### Plotting Diff-in-Diffs

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
data_merge = pd.read_csv('../20_analysis/big_merge.csv')


In [3]:
#data_merge = pd.read_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/20_analysis/big_merge.csv')
data_merge = data_merge.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
data_merge.head()

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year
0,male,Abbotabad,urban,0,0,NaN,2004
1,male,Abbotabad,rural,60,55,0.916667,2004
2,male,Attock,urban,0,0,NaN,2004
3,male,Attock,rural,64,60,0.937500,2004
4,male,Awaran,urban,0,0,NaN,2004


In [4]:
# dropping NAs
data_merge = data_merge.dropna(axis = 0)

In [5]:
data_merge['region'].replace('2', 'rural', inplace = True)
data_merge['region'].replace('1', 'urban', inplace = True)

In [6]:
data_merge.loc[data_merge.year == 2006]

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year
716,male,Abbotabad,rural,307,287,0.934853,2006
718,male,Attock,rural,369,349,0.945799,2006
720,male,Awaran,rural,289,207,0.716263,2006
722,male,Badin,rural,845,487,0.576331,2006
724,male,Bahawal Nagar,rural,777,633,0.814672,2006
...,...,...,...,...,...,...,...
1114,female,Thatta,rural,721,178,0.246879,2006
1116,female,Upper Dir,rural,675,314,0.465185,2006
1118,female,Vehari,rural,675,369,0.546667,2006
1120,female,Zhob,rural,489,163,0.333333,2006


In [7]:
data_post = data_merge[data_merge.year >= 2008]
data_pre = data_merge[data_merge.year < 2007]

In [8]:
data_pre.year.value_counts()

2004    246
2006    230
2005    218
Name: year, dtype: int64

In [9]:
# Mean enrollment rates
data_merge.groupby(['sex', 'year'])['rate_enrollment'].mean().reset_index()

,sex,year,rate_enrollment
0,female,2004,0.461533
1,female,2005,0.510109
2,female,2006,0.569063
3,female,2007,0.335922
4,female,2008,0.563482
5,female,2010,0.565069
6,female,2011,0.564087
7,female,2012,0.622112
8,female,2013,0.607155
9,female,2014,0.642135


In [13]:
## Nick's code for confidence bands 
def get_reg_fit(data, yvar, xvar, alpha=0.05,col="blue"):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color=col).encode(
        x=alt.X(xvar, axis=alt.Axis(title='year')),
        y=alt.X(yvar, axis=alt.Axis(title='')))
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color=col)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=""),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

In [14]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [15]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['red', 'blue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before = alt.Chart(
    data_merge[data_merge["sex"] == 'female'], title="Enrollment Trends Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female = before.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line = get_reg_fit(
    data_pre[data_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="red"
)

fit, female_post_line = get_reg_fit(
    data_post[data_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="red"
)



## Male Trends
alt.data_transformers.disable_max_rows()
before = alt.Chart(
    data_merge[data_merge["sex"] == 'male'], title="Enrollment Trends Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male = before.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line = get_reg_fit(
    data_pre[data_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="blue"
)

fit, male_post_line = get_reg_fit(
    data_post[data_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="blue"
)

plots= base_female + female_pre_line + female_post_line + base_male + male_pre_line + male_post_line + sep_line
plots

PatsyError: Error evaluating factor: NameError: name 'year' is not defined
    rate_enrollment ~ year
                      ^^^^

### Rural Data Diff in Diff

In [ ]:
# Rural Areas
rural = data_merge.loc[(data_merge.region == 'rural')]

In [ ]:
rural_post = rural[rural.year >= 2008]
rural_pre = rural[rural.year <= 2006]

In [ ]:
rural_pre.year.value_counts()

In [ ]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [ ]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['palevioletred', 'cornflowerblue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before_rural = alt.Chart(
    rural[rural["sex"] == 'female'], title="Enrollment Trends in Rural Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female_rural= before_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line_rural = get_reg_fit(
    rural_pre[rural_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)

fit, female_post_line_rural = get_reg_fit(
    rural_post[rural_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)



## Male Trends
alt.data_transformers.disable_max_rows()
after_rural = alt.Chart(
    rural[rural["sex"] == 'male'], title="Enrollment Trends in Rural Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male_rural = after_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line_rural = get_reg_fit(
    rural_pre[rural_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

fit, male_post_line_rural = get_reg_fit(
    rural_post[rural_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

plots_rural = base_female_rural + female_pre_line_rural + female_post_line_rural + base_male_rural + male_pre_line_rural + male_post_line_rural + sep_line
plots_rural

### Urban Areas Diff in Diff

In [ ]:
# Urban Areas
urban = data_merge.loc[data_merge.region == 'urban']

In [ ]:
urban_post = urban[urban.year >= 2008]
urban_pre = urban[urban.year <= 2006]

In [ ]:
urban_pre.year.value_counts()

In [ ]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line_urban = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [ ]:
# Enrolment by Gender

legend = alt.Chart(data_merge).transform_calculate(f= "'female'", m= "'male'")
scale = alt.Scale(domain=["Female", "Male"], range=['palevioletred', 'cornflowerblue'])

## Female Trends
alt.data_transformers.disable_max_rows()
before_urban = alt.Chart(
    urban[urban["sex"] == 'female'], title="Enrollment Trends in Urban Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('female:N', scale=scale, title=''))

base_female_urban= before_urban.transform_regression("year", "rate_enrollment").mark_line()

fit, female_pre_line_urban = get_reg_fit(
    urban_pre[urban_pre["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)

fit, female_post_line_urban = get_reg_fit(
    urban_post[urban_post["sex"] == 'female'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred"
)



## Male Trends
alt.data_transformers.disable_max_rows()
after_urban = alt.Chart(
    urban[urban["sex"] == 'male'], title="Enrollment Trends in Urban Pakistan"
).encode(x="year", y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)), color=alt.Color('male:N', scale=scale, title=''))

base_male_urban = after_rural.transform_regression("year", "rate_enrollment").mark_line()

fit, male_pre_line_urban = get_reg_fit(
    urban_pre[urban_pre["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

fit, male_post_line_urban = get_reg_fit(
    urban_post[urban_post["sex"] == 'male'],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue"
)

plots_urban = base_female_urban + female_pre_line_urban + female_post_line_urban + base_male_urban + male_pre_line_urban + male_post_line_urban + sep_line_urban
plots_urban

## Women diff-in-diff for taliban controlled cities vs not

In [ ]:
women = data_merge.loc[data_merge.sex == 'female']
women.region.value_counts()

In [ ]:
taliban_dominance_list = [
    "South Waziristan",
    "North Waziristan",
    "Orakzai",
    "Kurram",
    "Khyber",
    "Mohmand",
    "Bajur",
    "Darra Adamkhel",
    "Swat",
    "Upper Dir",
    "Lower Dir",
    "Bannu",
    "Lakki Marwat",
    "Tank",
    "Peshawar",
    "Dera Ismail Khan",
    "Mardan",
    "Charsadda",
    "Kohat",
]

In [ ]:
## Creating taliban dominated indicator
women['taliban_indic'] = np.where(women['subprovince'].isin(taliban_dominance_list), 1, 0)
women.head()

In [ ]:
# Separating Line
data = pd.DataFrame({"a": [2007]})
sep_line_taliban = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x=alt.X("a:Q", title="")))

In [ ]:
women_pre = women[women.year <= 2006]
women_post = women[women.year >= 2008]

In [ ]:
# Women Enrolment in Taliban controlled regions and otherwise

legend = alt.Chart(data_merge).transform_calculate(f="1", m="0")
scale = alt.Scale(
    domain=["Taliban-Controlled Regions", "Non-Taliban-controlled Regions"],
    range=["palevioletred", "cornflowerblue"],
)

## Female Trends
alt.data_transformers.disable_max_rows()
tal_base = alt.Chart(
    women[women["taliban_indic"] == 1], title="Enrollment Trends for Women in Pakistan"
).encode(
    x="year",
    y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)),
    color=alt.Color("female:N", scale=scale, title=""),
)

base_taliban = tal_base.transform_regression("year", "rate_enrollment").mark_line()

fit, taliban_pre_line = get_reg_fit(
    women_pre[women_pre["taliban_indic"] == 1],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred",
)

fit, taliban_post_line = get_reg_fit(
    women_post[women_post["taliban_indic"] == 1],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred",
)


## Male Trends
alt.data_transformers.disable_max_rows()
no_tal_base = alt.Chart(
    women[women["taliban_indic"] == 0], title="Enrollment Trends for Women in Pakistan"
).encode(
    x="year",
    y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)),
    color=alt.Color("male:N", scale=scale, title=""),
)

base_other = no_tal_base.transform_regression("year", "rate_enrollment").mark_line()

fit, other_pre_line = get_reg_fit(
    women_pre[women_pre["taliban_indic"] == 0],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue",
)

fit, other_post_line = get_reg_fit(
    women_post[women_post["taliban_indic"] == 0],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue",
)

plots_taliban_region = (
    base_taliban
    + taliban_pre_line
    + taliban_post_line
    + base_other
    + other_pre_line
    + other_post_line
    + sep_line_taliban
)
plots_taliban_region


In [ ]:
## Comparing women in taliban regions to other rural regions
women.head()
women_rural = women.loc[women.region == 'rural']
women_rural.head()

In [ ]:
women_rural_pre = women_rural[women_rural.year <= 2006]
women_rural_post = women_rural[women_rural.year >= 2008]

In [ ]:
# women_rural Enrolment in Taliban controlled regions and other RURAL regions

legend = alt.Chart(data_merge).transform_calculate(f="1", m="0")
scale = alt.Scale(
    domain=["Taliban-Controlled Regions", "Non-Taliban-controlled Regions"],
    range=["palevioletred", "cornflowerblue"],
)

## Female Trends
alt.data_transformers.disable_max_rows()
tal_base_ = alt.Chart(
    women_rural[women_rural["taliban_indic"] == 1], title="Enrollment Trends for Women in Rural Pakistan"
).encode(
    x="year",
    y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)),
    color=alt.Color("female:N", scale=scale, title=""),
)

base_taliban_ = tal_base_.transform_regression("year", "rate_enrollment").mark_line()

fit, taliban_pre_line_ = get_reg_fit(
    women_rural_pre[women_rural_pre["taliban_indic"] == 1],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred",
)

fit, taliban_post_line_ = get_reg_fit(
    women_rural_post[women_rural_post["taliban_indic"] == 1],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="palevioletred",
)


## Male Trends
alt.data_transformers.disable_max_rows()
other_base = alt.Chart(
    women_rural[women_rural["taliban_indic"] == 0], title="Enrollment Trends for Women in Rural Pakistan"
).encode(
    x=alt.X('year'),
    y=alt.Y("rate_enrollment", title="Rate of Enrollment", scale=alt.Scale(zero=False)),
    color=alt.Color("male:N", scale=scale, title=""),
)

base_other_ = other_base.transform_regression("year", "rate_enrollment").mark_line()

fit, other_pre_line_ = get_reg_fit(
    women_rural_pre[women_rural_pre["taliban_indic"] == 0],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue",
)

fit, other_post_line_ = get_reg_fit(
    women_rural_post[women_rural_post["taliban_indic"] == 0],
    yvar="rate_enrollment",
    xvar="year",
    alpha=0.05,
    col="cornflowerblue",
)

plots_taliban_rural_region = (
    base_taliban_
    + taliban_pre_line_
    + taliban_post_line_
    + base_other_
    + other_pre_line_
    + other_post_line_
    + sep_line_taliban
)


plots_taliban_rural_region


In [ ]:
axis_labels = (
    "datum.label == 0 ? 'Infrequent'
    : datum.label == 0.5 ? 'Average'
    : 'Frequent'"
)
alt.X('x:Q',axis=alt.Axis(labelExpr=axis_labels))